# Lab | Agent & Vector store

**Change the state union dataset and replicate this lab by updating the prompts accordingly.**

One such dataset is the [sonnets.txt](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/sonnets.txt) dataset or any other data of your choice from the same git.

# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [1]:
#%pip install chromadb

  Using cached opentelemetry_api-1.27.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached opentelemetry_sdk-1.27.0-py3-none-any.whl.metadata (1.5 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached bcrypt-4.2.0-cp39-abi3-win_amd64.whl.metadata (9.9 kB)
  Using cached typer-0.12.5-py3-none-any.whl.metadata (15 kB)
  Using cached tomli-2.0.2-py3-none-any.whl.metadata (10.0 kB)
  Using cached google_auth-2.35.0-py2.py3-none-any.whl.metadata (4.7 kB)
  Using cached Deprecated-1.2.14-py2.py3-none-any.whl.metadata (5.4 kB)
  Using cached opentelemetry_semantic_conventions-0.48b0-py3-none-any.whl.metadata (2.4 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached c

In [1]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

creating an instance of an OpenAI language model (LLM) with a specific configuration for how the model generates responses

In [3]:
llm = OpenAI(temperature=0)

finding the file path of a specific document (state_of_the_union.txt) based on the current file system's structure, and it is doing so dynamically by traversing through the directory path.

In [29]:
from pathlib import Path

relevant_parts = []
for p in Path(".").absolute().parts:
    relevant_parts.append(p)
    if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
        break
doc_path = str(Path(*relevant_parts) / "sonnets.txt")

loading a document, splitting it into chunks, creating embeddings for each chunk, and then storing these embeddings in a Chroma vector store for later retrieval. 

In [31]:
loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings, collection_name="sonnets")

creates a Retrieval Question-Answering (QA) chain using the previously defined language model (LLM) and the vector store (Chroma) with embeddings.

In [32]:
state_of_union = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

This line imports the WebBaseLoader class from the langchain_community.document_loaders module. Here's what it does:

from langchain_community.document_loaders import WebBaseLoader:
WebBaseLoader is a class that allows you to load documents from web-based sources. It's part of the LangChain community extensions, which provides various utilities for working with external data.
The WebBaseLoader is useful for scraping or retrieving web pages and turning them into documents that can be used in downstream tasks, such as loading into vector stores, performing embeddings, or using them for retrieval-based QA systems.

In [33]:
from langchain_community.document_loaders import WebBaseLoader

using the WebBaseLoader class to load a web page into the LangChain pipeline for further processing.

In [34]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

 continues the process of loading web content, splitting it into chunks, embedding it into a vector store, and setting up a RetrievalQA system. 

In [10]:
#!pip install beautifulsoup4

In [35]:
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

Created a chunk of size 2130, which is longer than the specified 1000
Created a chunk of size 3187, which is longer than the specified 1000
Created a chunk of size 1256, which is longer than the specified 1000
Created a chunk of size 1051, which is longer than the specified 1000


## Create the Agent

importing modules necessary to create an agent using LangChain, specifically one that can interact with different tools, like a language model or vector store.

In [37]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

defines a list of tools that will be used by the agent. Each tool corresponds to a specific functionality, such as answering questions about the "State of the Union" or "Ruff (a Python linter)."

In [38]:
tools = [
    Tool(
        name="sonnets QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the Shakespeare Sonnets. Input should be a fully formed question.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
    ),
]

initializes the agent by specifying the tools, the language model, and the agent type, allowing the agent to act intelligently using the available tools.

In [39]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

how the agent is invoked to answer a specific query using the tools and language model it has been set up with

In [40]:
agent.invoke(
    "What is Shakespeare talking about in the first three sonnets?"
)



> Entering new AgentExecutor chain...
 I should use the sonnets QA System to answer this question.
Action: sonnets QA System
Action Input: "What is Shakespeare talking about in the first three sonnets?"
Observation:  Shakespeare is talking about the desire for procreation and the fear of mortality in the first three sonnets.
Thought: I should use the sonnets QA System to answer this question.
Action: sonnets QA System
Action Input: "What is the overall theme of the first three sonnets?"
Observation:  The overall theme of the first three sonnets is the speaker's desire for the subject's beauty to be preserved and immortalized.
Thought: I should use the sonnets QA System to answer this question.
Action: sonnets QA System
Action Input: "What is the rhyme scheme of the first three sonnets?"
Observation:  The rhyme scheme of the first three sonnets is ABAB CDCD EFEF GG.
Thought: I now know the final answer.
Final Answer: The first three sonnets of Shakespeare's sonnet collection focus on 

{'input': 'What is Shakespeare talking about in the first three sonnets?',
 'output': "The first three sonnets of Shakespeare's sonnet collection focus on the themes of desire for procreation, fear of mortality, and the preservation of beauty. The overall theme is the speaker's desire for the subject's beauty to be immortalized. The rhyme scheme is ABAB CDCD EFEF GG."}

In [16]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 Ruff and flake8 are both python linters, so they have similar functions. However, there may be specific reasons why one is preferred over the other.
Action: Ruff QA System
Action Input: "Why use ruff over flake8?"
Observation: 
There are a few reasons why someone might choose to use Ruff over Flake8:

1. Ruff has a larger rule set: Ruff implements over 800 rules, while Flake8 only implements around 200. This means that Ruff can catch more potential issues in your code.

2. Ruff is faster: Ruff is written in Rust, which is a faster language than Python. This means that Ruff can lint your code more quickly than Flake8.

3. Ruff has a built-in formatter: Ruff includes a formatter that can automatically fix some of the linting issues it finds. This can save you time and effort when cleaning up your code.

4. Ruff is compatible with Black: Ruff's formatter is designed to be compatible with Black, another popular code formatter. This means that you ca

{'input': 'Why use ruff over flake8?',
 'output': 'Ruff offers a larger rule set, faster performance, a built-in formatter, compatibility with Black, and support for newer versions of Python, making it a more comprehensive and efficient linter compared to Flake8.'}

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

ACTING AS A ROUTER:  In this mode, the agent doesn't perform additional reasoning or actions after querying a tool

In [41]:
tools = [
    Tool(
        name="Sonnets QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the Shakespeare Sonnets. Input should be a fully formed question.",
        return_direct=True,
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [42]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [43]:
agent.invoke(
    "What is the main theme in the sonnets?"
)



> Entering new AgentExecutor chain...
 The main theme in the sonnets could vary depending on the interpretation of the reader.
Action: Sonnets QA System
Action Input: "What is the main theme in the sonnets?"
Observation:  The main theme in the sonnets is the power and inevitability of time and its effects on beauty and mortality.


> Finished chain.


{'input': 'What is the main theme in the sonnets?',
 'output': ' The main theme in the sonnets is the power and inevitability of time and its effects on beauty and mortality.'}

In [44]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 You should consider the differences between ruff and flake8 before making a decision.
Action: Ruff QA System
Action Input: "What are the differences between ruff and flake8?"
Observation:  Ruff has a larger rule set and does not support custom lint rules, while Flake8 supports plugins and allows for custom rules. Ruff also has the ability to automatically fix its own lint violations.


> Finished chain.


{'input': 'Why use ruff over flake8?',
 'output': ' Ruff has a larger rule set and does not support custom lint rules, while Flake8 supports plugins and allows for custom rules. Ruff also has the ability to automatically fix its own lint violations.'}

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

MULTI-HOP:  The agent allows the agent to process questions that require knowledge from multiple vector stores or sources. The agent can "hop" between the different tools (here, the State of Union QA System and the Ruff QA System) to retrieve and combine relevant information.

In [45]:
tools = [
    Tool(
        name="Sonnets QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the Shakespeare Sonnets. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
]

In [46]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [47]:
agent.invoke(
    "How can we lint Shakespeare's sonnets with ruff?"
)



> Entering new AgentExecutor chain...
 We can use ruff to check for any errors or inconsistencies in the sonnets.
Action: Ruff QA System
Action Input: "How can we lint Shakespeare's sonnets with ruff?"
Observation:  I don't know, as Ruff is a linter for Python code and not for literature.
Thought: Maybe we can use a different tool specifically designed for analyzing literature.
Action: Sonnets QA System
Action Input: "How can we analyze Shakespeare's sonnets?"
Observation: 
Shakespeare's sonnets can be analyzed by examining the themes, structure, language, and literary devices used in each individual sonnet. It is also important to consider the overall structure of the collection, as well as the historical context in which they were written. Additionally, analyzing the relationships between the speaker, the subject, and the intended audience can provide insight into the meaning and purpose of each sonnet. Close reading and interpretation of specific lines and phrases can also reveal 

{'input': "How can we lint Shakespeare's sonnets with ruff?",
 'output': "To analyze Shakespeare's sonnets, we can use a combination of tools such as close reading, historical context, and poetic devices. Ruff, a linter for Python code, is not suitable for analyzing literature."}